# Peak Flow

Run delineations for both the "series" and "parallel" approaches.

In [1]:
from pathlib import Path
import json
from statistics import mean
import math
from typing import List
from pprint import PrettyPrinter

# from arcpy import FeatureSet, RecordSet, Raster, Describe, CreateUniqueName, EnvManager, SpatialReference
# from arcpy.management import (
#     MakeFeatureLayer, 
#     SelectLayerByAttribute, 
#     GetCount, 
#     MakeTableView, 
#     BuildRasterAttributeTable,
#     Dissolve,
#     AddFields,
#     CreateFeatureclass
# )
# from arcpy.analysis import Buffer
# from arcpy.sa import Watershed, FlowLength, ZonalStatisticsAsTable, SetNull, IsNull
# from arcpy.da import SearchCursor, InsertCursor
# from arcpy.conversion import RasterToPolygon, JSONToFeatures

from dataclasses import dataclass, field, asdict
from marshmallow import Schema, fields, EXCLUDE, pre_load
from marshmallow_dataclass import class_schema

from tqdm.notebook import tqdm
import petl as etl

# import arcgis

In [2]:
pp = PrettyPrinter(indent=2).pprint

In [3]:
wd = Path(r"D:\Dropbox (CivicMapper)\Projects\202004-02 Cornell Modeling\3 - Production\tool outputs")
test_results_dir = wd / "c19 tests" / "test_results"

rainfall_rasters_config = Path(r"D:\Dropbox (CivicMapper)\Projects\202004-02 Cornell Modeling\3 - Production\tool outputs\c19 tests\noaa_rainfall\noaa_rainfall_rasters_config_ne.json")

inlets = wd / "c19 tests"  / "c19_snapped_points.shp"
dem = wd / "c19 tests" / "dem_clip_simple.tif"
fd = wd / "c19 tests" / "dem_clip_simple_flowdir.tif"
cn = wd / "c19 tests" / "curveno.tif"
slope = wd / "c19 tests" / "dem_slope.tif"

pour_point_field = "NAACC_ID"
out_catchment_polygons_simplify = False

In [10]:
test_points_json_path = Path(r'C:\Users\chris\dev\drainage\drainit\tests\points_after_delineation.json')

## Models:

In [4]:
@dataclass
class Rainfall:
    """storm frequency interval with rainfall values and rainfall-dependent 
    analytical results from the calculators
    """

    freq: int = None
    dur: str = None
    value: float = None
        
RainfallSchema = class_schema(Rainfall)

In [5]:
@dataclass
class Shed:
    """Characteristics of a single point's contributing area
    """
    # unique id field, derived from the outlet point; the value from the
    # "pour_point_field". For NAACC-based culvert modeling, this is the
    # NAACC Naacc_Culvert_Id field
    uid: str = None
    
    # a group id field. non-unique ID field that indicates groups of related
    # outlets. Used primarily for NAACC-based culvert modeling, this is the
    # NAACC Survey_Id field
    group_id: str = None

    # characteristics used for calculating peak flow
    area_sqkm: float = None# <area of inlet's catchment in square km>
    avg_slope_pct: float = None # <average slope of DEM in catchment>
    avg_cn: float = None # <average curve number in the catchment>
    max_fl: float = None # <maximum flow length in the catchment>
    rainfall: List[Rainfall] = None # average rainfall in the catchment, for storm events

    # geometries
    inlet_geom: dict = None
    shed_geom: dict = None
    
    # for recording the location of intermediate geospatial output files
    shed_polygon_filepath: str = None
    shed_raster_filepath: str = None
        
ShedSchema = class_schema(Shed)

In [6]:
@dataclass
class Point:
    """Basic model for points used as source delineations for peak-flow-calcs;
    minimal attributes required.
    """

    # unique id field, derived from the outlet point; the value from the
    # "pour_point_field". For NAACC-based culvert modeling, this is the
    # NAACC Naacc_Culvert_Id field
    uid: str

    lat: float = None
    lng: float = None
    spatial_ref_code: int = None

    # a group id field. non-unique ID field that indicates groups of related
    # outlets. Used primarily for NAACC-based culvert modeling, this is the
    # NAACC Survey_Id field
    group_id: str = None

    # optionally extend with NAACC & capacity attributes
    # naacc: NaaccCulvert = None
    # capacity: Capacity = None
    naacc: dict = None
    capacity: dict = None

    # optionally extend with the Shed and its characteristics
    shed: Shed = None

    # rainfall frequency-based analytical results for the point
    analytics: List[Rainfall] = field(default_factory=list)

    # flags, errors, and notes
    include: bool = True
    validation_errors: dict = field(default_factory=dict)
    notes: list = field(default_factory=list)

    # place to store the raw input
    raw: dict = None

## Read the Points from our test `JSON`


In [7]:
PointSchema = class_schema(Point)

In [11]:
with open(test_points_json_path) as fp:
    test_points = json.load(fp)
test_points

{'points': [{'shed': {'area_sqkm': 3198.260236505569,
    'avg_slope_pct': 16.102196872234344,
    'rainfall': [{'freq': 1, 'value': 59.74079999999999, 'dur': '24hr'},
     {'freq': 2, 'value': 74.1172, 'dur': '24hr'},
     {'freq': 5, 'value': 97.6122, 'dur': '24hr'},
     {'freq': 10, 'value': 117.1194, 'dur': '24hr'},
     {'freq': 25, 'value': 143.9672, 'dur': '24hr'},
     {'freq': 50, 'value': 163.80459999999997, 'dur': '24hr'},
     {'freq': 100, 'value': 185.3438, 'dur': '24hr'},
     {'freq': 200, 'value': 211.63279999999997, 'dur': '24hr'},
     {'freq': 500, 'value': 251.94259999999997, 'dur': '24hr'},
     {'freq': 1000, 'value': 286.61359999999996, 'dur': '24hr'}],
    'shed_raster_filepath': 'in_memory/catchment_70978.tif',
    'avg_cn': 77.75,
    'max_fl': 88.260292053223,
    'uid': '70978',
    'shed_geom': None,
    'shed_polygon_filepath': 'in_memory\\basin70978_dissolved',
    'inlet_geom': None,
    'group_id': None},
   'analytics': [],
   'capacity': None,
   's

In [9]:
points = [PointSchema().load(p) for p in test_points['points']]
points[0]

Point(uid='70978', lat=4690423.416399388, lng=618918.2765349224, spatial_ref_code=26918, group_id=None, naacc=None, capacity=None, shed=Shed(uid='70978', group_id=None, area_sqkm=3198.260236505569, avg_slope_pct=16.102196872234344, avg_cn=77.75, max_fl=88.260292053223, rainfall=[Rainfall(freq=1, dur='24hr', value=59.74079999999999), Rainfall(freq=2, dur='24hr', value=74.1172), Rainfall(freq=5, dur='24hr', value=97.6122), Rainfall(freq=10, dur='24hr', value=117.1194), Rainfall(freq=25, dur='24hr', value=143.9672), Rainfall(freq=50, dur='24hr', value=163.80459999999997), Rainfall(freq=100, dur='24hr', value=185.3438), Rainfall(freq=200, dur='24hr', value=211.63279999999997), Rainfall(freq=500, dur='24hr', value=251.94259999999997), Rainfall(freq=1000, dur='24hr', value=286.61359999999996)], inlet_geom=None, shed_geom=None, shed_polygon_filepath='in_memory\\basin70978_dissolved', shed_raster_filepath='in_memory/catchment_70978.tif'), analytics=[], include=True, validation_errors={}, notes